# Aims of the notebook

- Export pkl file to excel to count how many entries has entangled pop > X%

In [1]:
import pandas as pd
import numpy as np
import re, os
import json
import matplotlib.pyplot as plt
%matplotlib notebook

# Read Fully IDP data

In [2]:
fully_idp = pd.read_pickle('fully_idp.pkl')

In [3]:
fully_idp

,ped_id,number_of_chains,protein_name,length_in_ped,uniprot_acc,length_in_uniprot,disorder_region_from_mobi,pdb_region,length_of_idr,class,exp_method,number_of_entangled_frames,total_frames,note,entangled_population
341,PED00289,1.0,C-C motif chemokine 28,108.0,Q9NRJ3,127,"([1, 127],)","(20, 127)",[108],Fully IDP,NMR,20.0,20.0,NaN,1.000000
512,PED00418_A,NaN,"Membrane-associated guanylate kinase, WW and P...",129.0,Q96QZ7,1491,"([1, 15], [49, 61], [219, 294], [343, 354], [4...","(455, 580)","[0, 0, 0, 0, 104, 21, 0, 0, 0, 0, 0]",Fully IDP,NMR,20.0,20.0,"mismatch betweeb MobiDB, PED.\nEven MobiDB it ...",1.000000
202,PED00169,1.0,Isoform 3 of Microtubule-associated protein 2,467.0,P15146-3,467,no mobiDB info,"(1, 467)",467,Fully IDP,SAXS,4.0,4.0,manualy assigned,1.000000
511,PED00417,1.0,"Membrane-associated guanylate kinase, WW and P...",129.0,Q96QZ7,1491,"([1, 15], [49, 61], [219, 294], [343, 354], [4...","(455, 580)","[0, 0, 0, 0, 104, 21, 0, 0, 0, 0, 0]",Fully IDP,NMR,20.0,20.0,"mismatch betweeb MobiDB, PED.\nEven MobiDB it ...",1.000000
179,PED00145,1.0,Nuclear pore complex protein Nup153,114.0,P49790,1475,"([1, 666], [679, 721], [751, 794], [823, 850],...","(884, 993)","[0, 0, 0, 0, 110]",Fully IDP,SAXS,12621.0,18086.0,NaN,0.697833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,PED00244_A,NaN,FUS low complexity domain,61.0,P35637,526,"([1, 512],)","(37, 97)",[61],Fully IDP,NMR,0.0,20.0,NaN,0.000000
283,PED00244_B,NaN,FUS low complexity domain,61.0,P35637,526,"([1, 512],)","(37, 97)",[61],Fully IDP,NMR,0.0,20.0,NaN,0.000000
284,PED00244_C,NaN,FUS low complexity domain,61.0,P35637,526,"([1, 512],)","(37, 97)",[61],Fully IDP,NMR,0.0,20.0,NaN,0.000000
285,PED00244_D,NaN,FUS low complexity domain,61.0,P35637,526,"([1, 512],)","(37, 97)",[61],Fully IDP,NMR,0.0,20.0,NaN,0.000000


In [7]:
# select all short_IDR that have entangled conformation only.
fully_idp_entangled = fully_idp[fully_idp['entangled_population']>=0.5]

In [8]:
fully_idp_entangled

,ped_id,number_of_chains,protein_name,length_in_ped,uniprot_acc,length_in_uniprot,disorder_region_from_mobi,pdb_region,length_of_idr,class,exp_method,number_of_entangled_frames,total_frames,note,entangled_population
341,PED00289,1.0,C-C motif chemokine 28,108.0,Q9NRJ3,127,"([1, 127],)","(20, 127)",[108],Fully IDP,NMR,20.0,20.0,NaN,1.000000
512,PED00418_A,NaN,"Membrane-associated guanylate kinase, WW and P...",129.0,Q96QZ7,1491,"([1, 15], [49, 61], [219, 294], [343, 354], [4...","(455, 580)","[0, 0, 0, 0, 104, 21, 0, 0, 0, 0, 0]",Fully IDP,NMR,20.0,20.0,"mismatch betweeb MobiDB, PED.\nEven MobiDB it ...",1.000000
202,PED00169,1.0,Isoform 3 of Microtubule-associated protein 2,467.0,P15146-3,467,no mobiDB info,"(1, 467)",467,Fully IDP,SAXS,4.0,4.0,manualy assigned,1.000000
511,PED00417,1.0,"Membrane-associated guanylate kinase, WW and P...",129.0,Q96QZ7,1491,"([1, 15], [49, 61], [219, 294], [343, 354], [4...","(455, 580)","[0, 0, 0, 0, 104, 21, 0, 0, 0, 0, 0]",Fully IDP,NMR,20.0,20.0,"mismatch betweeb MobiDB, PED.\nEven MobiDB it ...",1.000000
179,PED00145,1.0,Nuclear pore complex protein Nup153,114.0,P49790,1475,"([1, 666], [679, 721], [751, 794], [823, 850],...","(884, 993)","[0, 0, 0, 0, 110]",Fully IDP,SAXS,12621.0,18086.0,NaN,0.697833
440,PED00366,1.0,Polar organizing protein Z,141.0,Q9A8N4,177,"([1, 133],)","(1, 133)",[133],Fully IDP,NMR,13.0,20.0,8 tag residues are disorder,0.650000
267,PED00232,1.0,Alpha-synuclein,140.0,P37840,140,"([1, 140],)","(1, 140)",[140],Fully IDP,NMR,5.0,8.0,NaN,0.625000


In [5]:
# export short IDR with entanglement to excel file
fully_idp_entangled.to_excel('fully_idp_entangled.xlsx')